# Lecture 2: Introduction to Feature Engineering

## Setup

In [1]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [2]:
articles = pd.read_csv('../data/articles.csv')
customers = pd.read_csv('../data/customers.csv')
sample_submisison = pd.read_csv('../data/sample_submission.csv')
transactions = pd.read_csv('../data/transactions_train.csv')

## The H&M Dataset

In [3]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [4]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [6]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [7]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"../data/customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"../data/transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"../data/articles_sample{sample_repr}.csv.gz", index=False)

0.001
0.01
0.05


In [8]:
articles_sample = pd.read_csv('../data/articles_sample01.csv.gz')
customers_sample = pd.read_csv('../data/customers_sample01.csv.gz')
transactions_sample = pd.read_csv('../data/transactions_sample01.csv.gz')

In [9]:
customers_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371 entries, 0 to 1370
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             1371 non-null   object 
 1   FN                      478 non-null    float64
 2   Active                  469 non-null    float64
 3   club_member_status      1362 non-null   object 
 4   fashion_news_frequency  1352 non-null   object 
 5   age                     1347 non-null   float64
 6   postal_code             1371 non-null   object 
dtypes: float64(3), object(4)
memory usage: 75.1+ KB


In [10]:
transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32655 entries, 0 to 32654
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   t_dat             32655 non-null  object 
 1   customer_id       32655 non-null  object 
 2   article_id        32655 non-null  int64  
 3   price             32655 non-null  float64
 4   sales_channel_id  32655 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ MB


## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [11]:
# If you'd rather use a sample. Uncomment the following code:
transactions = transactions_sample
customers = customers_sample
articles = articles_sample

In [12]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [13]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,640182001,0.035576,2,1
1,2018-09-20,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,633685001,0.067780,2,1
2,2018-09-20,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,522561002,0.084729,2,1
3,2018-09-20,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,608483002,0.013542,2,1
4,2018-09-20,0cc7e8c8bcbf1aade96932a0a5e0ce0efc37ee82e17c63...,456163021,0.028797,2,1


In [14]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [15]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,640182001,0.035576,2,1
1,2018-09-20,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,633685001,0.067780,2,1
2,2018-09-20,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,522561002,0.084729,2,1
3,2018-09-20,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,608483002,0.013542,2,1
4,2018-09-20,0cc7e8c8bcbf1aade96932a0a5e0ce0efc37ee82e17c63...,456163021,0.028797,2,1


In [16]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [17]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

32655


In [18]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [19]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [20]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2019-02-16,d7658aaee939414d31bab252cc8505b6b7201e73e07e01...,785090001,0.0355763,2,0
1,2020-07-22,b39ccfdf7b780e3e619180edd366781df92e0346b22ec3...,790006003,0.0948983,2,0
2,2020-03-10,2cac4e7ad9c4680545474ea76b5b4f2defd041d169cca7...,583534014,0.0169322,1,0
3,2018-12-30,5ca77d0884911fb41fdb582dbda9319301c623736f1729...,656763006,0.0173898,1,0
4,2018-11-07,8003a273320ab206fc4418ce89174bc4f47193c6165ce1...,324502001,0.0101525,2,0


In [21]:
neg_transactions.shape

(35920, 6)

In [22]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [23]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [24]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65310 entries, 0 to 65309
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   t_dat                         65310 non-null  object 
 1   customer_id                   65310 non-null  object 
 2   article_id                    65310 non-null  object 
 3   price                         65310 non-null  object 
 4   sales_channel_id              65310 non-null  object 
 5   ordered                       65310 non-null  object 
 6   FN                            26050 non-null  float64
 7   Active                        25689 non-null  float64
 8   club_member_status            65087 non-null  object 
 9   fashion_news_frequency        64684 non-null  object 
 10  age                           64650 non-null  float64
 11  postal_code                   65310 non-null  object 
 12  product_code                  65310 non-null  int64  
 13  p

### Basic Preprocessing
Some very basic preprocessing.

In [25]:
# adding features: product_type (labeled), garment_group_no (labeled), combined with perceived_colour_value_name 

# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered', 'product_type_no', 'garment_group_no', 'perceived_colour_value_name']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,product_type_no,garment_group_no,perceived_colour_value_name
0,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,30.0,640182001,2,0.0355763,1,265,1013,Light
1,cc2909d3a827ff04e00466ea9da55ae50df1f7fc604d68...,57.0,640182001,2,0.0304915,1,265,1013,Light
2,cc2909d3a827ff04e00466ea9da55ae50df1f7fc604d68...,57.0,640182001,2,0.0304915,1,265,1013,Light
3,01271359dce99de537d6a3fca3c4dfd3732b455ec5085d...,30.0,633685001,2,0.0677797,1,265,1013,Dusty Light
4,ad65fce358652df068d272c5a3916bd5e8e083fa265dac...,18.0,633685001,2,0.0677797,0,265,1013,Dusty Light


In [26]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
p_type_encoder = preprocessing.LabelEncoder()
group_no_encoder = preprocessing.LabelEncoder()
color_name_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [27]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])
transactions_processed['garment_group_no'] = group_no_encoder.fit_transform(transactions_processed['garment_group_no'])
transactions_processed['product_type_no'] = p_type_encoder.fit_transform(transactions_processed['product_type_no'])
transactions_processed['perceived_colour_value_name'] = color_name_encoder.fit_transform(transactions_processed['perceived_colour_value_name'])

In [28]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['00bc9ab2554d0a51845fe034c2abb092897c8599cbf6726e0964969b04a79ec8'],
      dtype=object)

In [29]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,product_type_no,garment_group_no,perceived_colour_value_name
0,4,30.0,4182,2,0.0355763,1,53,11,3
1,1102,57.0,4182,2,0.0304915,1,53,11,3
2,1102,57.0,4182,2,0.0304915,1,53,11,3
3,4,30.0,3876,2,0.0677797,1,53,11,2
4,927,18.0,3876,2,0.0677797,0,53,11,2


In [30]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [31]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [32]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,product_type_no,garment_group_no,perceived_colour_value_name,sales_channel_id_1,sales_channel_id_2
0,4,30.0,4182,0.035576,1,53,11,3,0,1
1,1102,57.0,4182,0.030492,1,53,11,3,0,1
2,1102,57.0,4182,0.030492,1,53,11,3,0,1
3,4,30.0,3876,0.067780,1,53,11,2,0,1
4,927,18.0,3876,0.067780,0,53,11,2,0,1


In [33]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [34]:
X_train.head()

,customer_id,age,article_id,price,product_type_no,garment_group_no,perceived_colour_value_name,sales_channel_id_1,sales_channel_id_2
63853,126,23.0,15674,0.016932,2,15,3,0,1
63826,1154,28.0,2820,0.012186,2,15,3,0,1
61318,400,33.0,10693,0.050831,3,16,1,0,1
21913,219,57.0,10358,0.022017,53,11,2,0,1
49861,254,23.0,6470,0.010153,84,1,1,1,0


In [35]:
y_train.head()

63853    1
63826    0
61318    0
21913    1
49861    1
Name: ordered, dtype: int64

## Baseline Model

In [ ]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [ ]:
baseline.predict_proba(X_test)

In [ ]:
y_test

In [ ]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

In [ ]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
predictions

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).
